In [1]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"
!wget -O visa-outlier-clusters.csv "https://www.dropbox.com/scl/fi/duwtwt64uzv504if590sf/visa_outliers_US.csv?rlkey=6nictvov5pjaeue3yatz7m83k&dl=0"

--2023-10-12 17:12:27--  https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbb9ebe35f10aa8836fb001740a.dl-eu.dropboxusercontent.com/cd/0/inline/CFefwgSE3xW4d3EBhNLhhuccuhS_2SNh-_l7k-d_aJwcEMLMUj3scei-qk25k2IGsv29ftJSz2CUcWFeP1OnjnYSGvutPtDN_GvTlEkqwRR0_38vg9TWws-waI6hOauaTSz6AU9ilQ3KNzxfDN33p8Lc/file?dl=1# [following]
--2023-10-12 17:12:28--  https://ucbb9ebe35f10aa8836fb001740a.dl-eu.dropboxusercontent.com/cd/0/inline/CFefwgSE3xW4d3EBhNLhhuccuhS_2SNh-_l7k-d_aJwcEMLMUj3scei-qk25k2IGsv29ftJSz2CUcWFeP1OnjnYSGvutPtDN_GvTlEkqwRR0_38vg9TWws-waI6hOauaTSz6AU9ilQ3KNzxfDN33p8Lc/file?dl=1
Resolving ucbb9ebe35f10aa8836fb001740a.dl-eu.dropboxusercontent.com (ucbb9ebe35f10aa8836fb001740a.dl-eu.dro

In [ ]:
#!unzip wikitext-filtered-full.zip
#!unzip wikitext-filtered-10k.zip

Archive:  wikitext-filtered-full.zip
replace wikitext-filtered-full/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  wikitext-filtered-10k.zip
replace wikitext-filtered-10k/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install datasets
import datasets

In [ ]:
from datasets import load_dataset, Dataset

def load_dataset():
  wikitext_small = "wikitext-filtered-10k"
  wikitext_large = "wikitext-filtered-full"

  dataset_small = Dataset.load_from_disk(wikitext_small)
  dataset_large = Dataset.load_from_disk(wikitext_large)
  print("wikitext_small: {} docs, wikitext_large: {} docs".format(len(dataset_small), len(dataset_large)))
  return dataset_small, dataset_large

wikitext_small, wikitext_large = load_dataset()

wikitext_small: 10000 docs, wikitext_large: 859955 docs


Step 1:

In [ ]:
!pip install nltk

cleaning dataset

In [ ]:

#Download stop words from the library 'NLTK'

import nltk
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))



[nltk_data] Downloading package stopwords to /home/marc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import regex as re

# Removing characters non alphabetic
wikitext_small = [re.sub("[^A-Za-z']+", " ", sentence["text"]) for sentence in wikitext_small]
wikitext_large  = [re.sub("[^A-Za-z']+", " ", sentence["text"]) for sentence in wikitext_large]

In [ ]:
import gensim

#tokenize the words of the two differents datasets
wikitext_small_tokenized = [[token for token in gensim.utils.tokenize(sentence.lower(),  deacc = True) if not token in stop_words]
                             for sentence in wikitext_small]

wikitext_large_tokenized = [[token for token in gensim.utils.tokenize(sentence.lower(),  deacc = True) if not token in stop_words]
                             for sentence in wikitext_large]

step 2:

In [ ]:
#define hyperparameters variables

context_window = 5 # The distance of the words to take in account     

vector_size = 50 #how many dimensions is going to have the resultant vector of each word. This vector capture the relationships between words

# note: in the coursework pdf appears as 'iter', but it is deprecated and has been changed to 'epochs'.
epochs = 5  #how many times the model will go over the entire dataset.

min_count = 5  #how many times has the word appears in the dataset to to take in account.

In [1]:
from gensim.models import Word2Vec
try:
    small_df_model = Word2Vec.load('small_df.model')
    large_df_model = Word2Vec.load('large_df_model')
except:
#training models
    print("models not found")
    small_df_model = Word2Vec(sentences=wikitext_small_tokenized,vector_size=vector_size, window=context_window, min_count=min_count, epochs=epochs)
    small_df_model.save("small_df.model")
    large_df_model = Word2Vec(sentences=wikitext_large_tokenized, vector_size=vector_size, window=context_window, min_count=min_count, epochs=epochs)
    large_df_model.save("large_df_model")

STEP 3:

In [127]:
#crearting a function to print the cousines similarities
def print_cousine_similarities(first_word, second_word):
    print(f"{first_word}/{second_word}  \n", 
          "\tsmall dataset:", f"{small_df_model.wv.similarity(first_word, second_word)}\n",
            "\tlarge datset:", f"{large_df_model.wv.similarity(first_word, second_word)}\n")

In [5]:
print_cousine_similarities("plane", "car")
print_cousine_similarities("planet", "sun")
print_cousine_similarities("cup", "article")
print_cousine_similarities("sugar", "approach")

plane/car  
 	small dataset: 0.9873542785644531
 	large datset: 0.5832024216651917

planet/sun  
 	small dataset: 0.993909478187561
 	large datset: 0.6373756527900696

cup/article  
 	small dataset: 0.6564936637878418
 	large datset: 0.05309109389781952

sugar/approach  
 	small dataset: 0.9856008291244507
 	large datset: -0.21134068071842194



other words similarities

In [128]:
print_cousine_similarities("car", "truck")
print_cousine_similarities("love", "sex")
large_df_model.wv.most_similar(positive=["king", "woman"], negative=["male"])

car/truck  
 	small dataset: 0.9762018322944641
 	large datset: 0.8513953685760498

love/sex  
 	small dataset: 0.7270536422729492
 	large datset: 0.5573849678039551



[('prince', 0.6806843876838684),
 ('tsar', 0.6511449813842773),
 ('son', 0.6379523277282715),
 ('regent', 0.637367844581604),
 ('walsingham', 0.6260783076286316),
 ('deathbed', 0.6223783493041992),
 ('isabeau', 0.6184468269348145),
 ('hardinge', 0.6021685004234314),
 ('deed', 0.5984432697296143),
 ('throne', 0.5981664657592773)]

STEP 4:     

In [107]:

from scipy.stats import spearmanr
import pandas as pd

wordsim353_df = pd.read_csv('combined.csv')
wordsim353_df.head()

,Word 1,Word 2,Human (mean)
0,love,sex,6.77
1,tiger,cat,7.35
2,tiger,tiger,10.00
3,book,paper,7.46
4,computer,keyboard,7.62


In [111]:
#function to add the cousine correlation in the dataset
import numpy as np

def add_cousin_correlation(df : pd.DataFrame, name_label: str, model: Word2Vec):
    score = []
    missing_values = 0
    for index, row in df.iterrows():
        if row["Word 1"]  in model.wv.index_to_key and row["Word 2"] in model.wv.index_to_key:
            score.append( model.wv.similarity(row["Word 1"], row["Word 2"]))
        else:
            missing_values += 1
            score.append(np.nan)
    df[name_label] = score
    print(f"missing values of {name_label}: {missing_values}")

add_cousin_correlation(wordsim353_df, "small_df", small_df_model )
add_cousin_correlation(wordsim353_df, "large_df", large_df_model)

353
missing values of small_df: 101
missing values of large_df: 18


In [121]:
#clean the df
clean_small_wordsim353_df = wordsim353_df.dropna(subset=['small_df'], inplace=False)
clean_large_wordsim353_df = wordsim353_df.dropna(subset=['large_df'], inplace=False)

,Word 1,Word 2,Human (mean),small_df,large_df
0,love,sex,6.77,0.727054,0.557385
1,tiger,cat,7.35,0.994153,0.610930
2,tiger,tiger,10.00,1.000000,1.000000
3,book,paper,7.46,0.831142,0.580514
4,computer,keyboard,7.62,0.982206,0.422520


In [124]:
#calculate the spearman correlation

small_correlation = spearmanr(clean_small_wordsim353_df['Human (mean)'], clean_small_wordsim353_df["small_df"])
large_correlation = spearmanr(clean_large_wordsim353_df['Human (mean)'], clean_large_wordsim353_df["large_df"])
small_correlation, large_correlation

(SignificanceResult(statistic=0.10142118986984747, pvalue=0.1082406979417225),
 SignificanceResult(statistic=0.627196693222107, pvalue=4.979199074045186e-38))